In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train / 255.
X_test = X_test / 255.
X_train = X_train.reshape(*X_train.shape,1)
X_test = X_test.reshape(*X_test.shape,1)

In [11]:
cnn = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (28,28,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
    ]
)

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 7744)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                495680    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 515,146
Trainable params: 515,146
Non-trainable params: 0
________________________________________________

In [12]:
epochs = 5

cnn.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [13]:
cnn.fit(X_train, y_train, epochs = epochs)

2022-05-15 11:27:18.994544: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.
2022-05-15 11:27:19.413108: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
  12/1875 [..............................] - ETA: 29s - loss: 1.8636 - accuracy: 0.3229

2022-05-15 11:27:20.161802: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17981568 exceeds 10% of free system memory.
2022-05-15 11:27:20.162625: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17981568 exceeds 10% of free system memory.
2022-05-15 11:27:20.190293: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17981568 exceeds 10% of free system memory.
2022-05-15 11:27:20.190561: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17981568 exceeds 10% of free system memory.


1875/1875 [==============================] - 26s 13ms/step - loss: 0.4097 - accuracy: 0.8528
Epoch 2/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2660 - accuracy: 0.9035
Epoch 3/5
1875/1875 [==============================] - 26s 14ms/step - loss: 0.2180 - accuracy: 0.9197
Epoch 4/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1819 - accuracy: 0.9325
Epoch 5/5
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1518 - accuracy: 0.9431


In [3]:
MODEL_DIRECTORY = "./tmp"

if not os.path.exists(MODEL_DIRECTORY):
    os.makedirs(MODEL_DIRECTORY)

In [4]:
version = 1

In [5]:
export_path = os.path.join(MODEL_DIRECTORY, str(version))
print(f"export path = {export_path}")

if os.path.isdir(export_path):
    raise ValueError(f"model with version {version} already exists")

tf.saved_model.save(
    cnn,
    export_path,
)

export path = ./tmp/1


In [30]:
!ls -l {export_path}

total 124
drwxr-xr-x 2 jovyan users     64 May 15 11:42 assets
-rw-r--r-- 1 jovyan users 126758 May 15 11:42 saved_model.pb
drwxr-xr-x 4 jovyan users    128 May 15 11:42 variables


In [6]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['conv2d_2_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_conv2d_2_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_3'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inp